<a href="https://colab.research.google.com/github/lucianosilva-github/compiladores/blob/main/COMPILADORES_AULA_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**COMPILADORES - AULA 1**

***Prof. Luciano Silva***

**Objetivos da aula:**
*   revisar os passos de compilação
*   apresentar o conceito de analisador léxico
*   implementar analisadores léxicos em Python, usando o módulo rply





**FASES DO PROCESSO DE COMPILAÇÃO**

As diversas fases do processo de compilação são mostradas na imagem abaixo:

<img src="https://www.researchgate.net/profile/Nuno-Oliveira-15/publication/266497079/figure/fig1/AS:295651775664128@1447500284302/Common-Compiler-Phases.png"> <img>

A partir de um programa-fonte (source), a primeira fase do processo de compilação é a quebra do programa em **tokens**, realizado através do **analisador léxico**. 

Uma vez identificados os tokens, o compilador precisa verificar se eles aparecem numa ordem determinada por uma **gramática**, processo que é realizado pelo **analisador sintático**. O produto do analisador sintático é a chamada **árvore de análise sintática** ou, abreviadamente, **árvore sintática**.

A partir da árvore sintática, o compilador utiliza o **analisador semântico**. Este analisador percorre a árvore sintática para verificar, por exemplo, se há algum erro de tipo, se as funções são chamada com os argumentos corretos, dentre outras verificações. 

Uma vez que o programa não tenha erros sintáticos e semânticos, o compilador começa as tarefas de geração de código. O **gerador de código intermediário** é o primeiro deles e, para este gerador, a máquina-alvo do código é considerada como tendo um número infinito de registradores. Isto facilita bastante a geração de código inicial.

A partir deste código intermediário, caímos em um problema de otimização: como transformar um código com infinitos registradores em um código que utilize um número finito de registradores. Esta é uma das tarefas do **otimizador de código**.

Finalmente, uma vez otimizado o código, podemos gerar o código final para a máquina-alvo utilizando o **gerador de código**. 


Queremos constuir um analisador e resolvedor de expressões aritméticas, com ou sem parênteses.

\<expression\> ::= NUMBER

               | \<expression\> "+" \<expression\>

               | \<expression\> "-" \<expression\>

               | \<expression\> "*" \<expression\>

               | \<expression\> "/" \<expression\>

               | "(" <expression> ")"

O primeiro passo é instalar o rply, um módulo para construir analisadores.



In [ ]:
!pip install rply

O segundo passo é construir o analisador léxico:

In [ ]:
from rply import LexerGenerator

lg = LexerGenerator()

lg.add('NUMBER', r'\d+')
lg.add('PLUS', r'\+')
lg.add('MINUS', r'-')
lg.add('MUL', r'\*')
lg.add('DIV', r'/')
lg.add('OPEN_PARENS', r'\(')
lg.add('CLOSE_PARENS', r'\)')

lg.ignore('\s+')

lexer = lg.build()

O terceiro passo é construir uma representação para a árvore sintática:

In [ ]:
from rply.token import BaseBox

class Number(BaseBox):
    def __init__(self, value):
        self.value = value

    def eval(self):
        return self.value

class BinaryOp(BaseBox):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOp):
    def eval(self):
        return self.left.eval() + self.right.eval()

class Sub(BinaryOp):
    def eval(self):
        return self.left.eval() - self.right.eval()

class Mul(BinaryOp):
    def eval(self):
        return self.left.eval() * self.right.eval()

class Div(BinaryOp):
    def eval(self):
        return self.left.eval() / self.right.eval()

O quarto passo é construir uma analisador sintático, baseado na gramática da linguagem:

In [ ]:
from rply import ParserGenerator

pg = ParserGenerator(
    # A list of all token names, accepted by the parser.
    ['NUMBER', 'OPEN_PARENS', 'CLOSE_PARENS',
     'PLUS', 'MINUS', 'MUL', 'DIV'
    ],
    # A list of precedence rules with ascending precedence, to
    # disambiguate ambiguous production rules.
    precedence=[
        ('left', ['PLUS', 'MINUS']),
        ('left', ['MUL', 'DIV'])
    ]
)

@pg.production('expression : NUMBER')
def expression_number(p):
    # p is a list of the pieces matched by the right hand side of the
    # rule
    return Number(int(p[0].getstr()))

@pg.production('expression : OPEN_PARENS expression CLOSE_PARENS')
def expression_parens(p):
    return p[1]

@pg.production('expression : expression PLUS expression')
@pg.production('expression : expression MINUS expression')
@pg.production('expression : expression MUL expression')
@pg.production('expression : expression DIV expression')
def expression_binop(p):
    left = p[0]
    right = p[2]
    if p[1].gettokentype() == 'PLUS':
        return Add(left, right)
    elif p[1].gettokentype() == 'MINUS':
        return Sub(left, right)
    elif p[1].gettokentype() == 'MUL':
        return Mul(left, right)
    elif p[1].gettokentype() == 'DIV':
        return Div(left, right)
    else:
        raise AssertionError('Oops, this should not be possible!')

parser = pg.build()

Como resultado, temos a árvore sintática. Percorrendo esta árvore, podemos calcular o valor da expressão.

In [ ]:
parser.parse(lexer.lex('1 + 1')).eval()

In [ ]:
parser.parse(lexer.lex('1 + 2 * 3')).eval()